In [38]:
from pathlib import Path
import pandas as pd
import os
import os.path

In [39]:
image_collection_path = Path("../Dataset/Indian")

In [40]:
jpg_paths = list(image_collection_path.glob(r"*/*_canny.jpg"))

In [41]:
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1],jpg_paths))

In [42]:
jpg_path_series = pd.Series(jpg_paths, name="PATH").astype(str)
label_series = pd.Series(labels, name="LABEL")

In [43]:
path_dataset = pd.concat([jpg_path_series, label_series], axis=1)
path_dataset = path_dataset.sample(frac=1).reset_index(drop=True)

In [44]:
print(path_dataset.head(-1))

                                     PATH LABEL
0      ..\Dataset\Indian\I\1121_canny.jpg     I
1       ..\Dataset\Indian\7\684_canny.jpg     7
2        ..\Dataset\Indian\L\36_canny.jpg     L
3       ..\Dataset\Indian\Y\970_canny.jpg     Y
4       ..\Dataset\Indian\L\815_canny.jpg     L
...                                   ...   ...
42560    ..\Dataset\Indian\Q\14_canny.jpg     Q
42561   ..\Dataset\Indian\V\073_canny.jpg     V
42562  ..\Dataset\Indian\I\1024_canny.jpg     I
42563   ..\Dataset\Indian\F\153_canny.jpg     F
42564  ..\Dataset\Indian\C\1160_canny.jpg     C

[42565 rows x 2 columns]


In [45]:
from sklearn.model_selection import train_test_split

In [46]:
train_paths, test_paths = train_test_split(path_dataset, train_size=0.8, random_state=333, shuffle=True)
print(train_paths.shape)
print(test_paths.shape)

(34052, 2)
(8514, 2)


# CNN

In [47]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [48]:
train_generator = ImageDataGenerator(rescale=1./255, zoom_range=0.5,shear_range=0.5, brightness_range=[0.6,1.0],rotation_range=35,height_shift_range=0.1,width_shift_range=0.1,vertical_flip=True, featurewise_std_normalization=False, fill_mode="nearest", validation_split=0.1)
test_generator = ImageDataGenerator(rescale=1./255)

In [49]:
train_set = train_generator.flow_from_dataframe(dataframe=train_paths, x_col="PATH", y_col="LABEL", batch_size=32, class_mode="categorical", color_mode="grayscale", subset="training")
print(train_set.image_shape)
validation_set = train_generator.flow_from_dataframe(dataframe=train_paths, x_col="PATH", y_col="LABEL", batch_size=32, class_mode="categorical", color_mode="grayscale", subset="validation")
print(validation_set.image_shape)
test_set = test_generator.flow_from_dataframe(dataframe=test_paths, x_col="PATH", y_col="LABEL", batch_size=32, class_mode="categorical", color_mode="grayscale", shuffle=False)
print(test_set.image_shape)

Found 30647 validated image filenames belonging to 35 classes.
(256, 256, 1)
Found 3405 validated image filenames belonging to 35 classes.
(256, 256, 1)
Found 8514 validated image filenames belonging to 35 classes.
(256, 256, 1)


In [50]:
import tensorflow as tf

In [51]:
MODEL_LOSS = "categorical_crossentropy"
MODEL_OPTIMIZER = "adam"
MODEL_METRICS = ["accuracy"]
INPUT_DIM = train_set.image_shape
OUTPUT_DIM = 35
early_stopper_callback = tf.keras.callbacks.EarlyStopping(monitor = "loss", patience = 3, mode = "min")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(monitor = "val_accuracy",
                                                      save_weights_only = True,
                                                      filepath = "./modelcheckpoints")

In [52]:
from tensorflow.keras.models import Sequential
from keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Conv2D, Dropout, Flatten, Dense

In [53]:
Model = Sequential()

Model.add(Conv2D(32,(3,3),activation="relu",input_shape=INPUT_DIM))
Model.add(MaxPooling2D((2,2),strides=2))

Model.add(Conv2D(32,(3,3),activation="relu",padding="same"))
Model.add(MaxPooling2D((2,2),strides=2))

Model.add(Conv2D(64,(3,3),activation="relu",padding="same"))
Model.add(MaxPooling2D((2,2),strides=2))

Model.add(Conv2D(64,(3,3),activation="relu",padding="same"))
Model.add(Conv2D(64,(3,3),activation="relu",padding="same"))
Model.add(Dropout(0.3))
Model.add(MaxPooling2D((2,2),strides=2))

Model.add(Conv2D(128,(3,3),activation="relu",padding="same"))
Model.add(Dropout(0.3))
Model.add(MaxPooling2D((2,2),strides=2))

Model.add(Flatten())
Model.add(Dense(256,activation="relu"))
Model.add(Dropout(0.5))
Model.add(Dense(OUTPUT_DIM,activation="softmax"))

In [56]:
Model.compile(optimizer=MODEL_OPTIMIZER,loss=MODEL_LOSS,metrics=MODEL_METRICS)

In [57]:
CNN_Model = Model.fit(train_set,
                      validation_data=validation_set,
                      callbacks=[early_stopper_callback,checkpoint_callback],
                      epochs=50)

Epoch 1/50
958/958 [==============================] - 834s 870ms/step - loss: 2.1397 - accuracy: 0.3757 - val_loss: 1.0125 - val_accuracy: 0.7777
Epoch 2/50
958/958 [==============================] - 814s 849ms/step - loss: 0.8370 - accuracy: 0.7385 - val_loss: 0.5023 - val_accuracy: 0.9016
Epoch 3/50
958/958 [==============================] - 814s 849ms/step - loss: 0.4979 - accuracy: 0.8375 - val_loss: 0.2920 - val_accuracy: 0.9465
Epoch 4/50
958/958 [==============================] - 832s 868ms/step - loss: 0.3665 - accuracy: 0.8837 - val_loss: 0.1580 - val_accuracy: 0.9709
Epoch 5/50
958/958 [==============================] - 818s 854ms/step - loss: 0.2948 - accuracy: 0.9051 - val_loss: 0.1268 - val_accuracy: 0.9791
Epoch 6/50
958/958 [==============================] - 797s 832ms/step - loss: 0.2525 - accuracy: 0.9156 - val_loss: 0.1143 - val_accuracy: 0.9783
Epoch 7/50
958/958 [==============================] - 822s 857ms/step - loss: 0.2198 - accuracy: 0.9283 - val_loss: 0.0981 -

In [ ]:
Model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [ ]:
Model.save("predictor_model")

ValueError: Model <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000022F0ABD4640> cannot be saved because the input shapes have not been set. Usually, input shapes are automatically determined from calling `.fit()` or `.predict()`. To manually set the shapes, call `model.build(input_shape)`.

In [ ]:
Grap_Data = pd.DataFrame(CNN_Model.history)
Grap_Data.plot()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(CNN_Model.history["accuracy"])
plt.plot(CNN_Model.history["val_accuracy"])
plt.ylabel("ACCURACY")
plt.legend()
plt.show()

NameError: name 'CNN_Model' is not defined

In [ ]:
plt.plot(CNN_Model.history["loss"])
plt.plot(CNN_Model.history["val_loss"])
plt.ylabel("LOSS")
plt.legend()
plt.show()

In [ ]:
Model_Results = Model.evaluate(test_set)
print("LOSS:  " + "%.4f" % Model_Results[0])
print("ACCURACY:  " + "%.2f" % Model_Results[1])